# Quick, Draw! Doodle Recognition Challenge

Firstly we import the nedded packages and libraries to run the python script.
We then show the directories in which the data we need are in.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import ast
from datetime import date, timedelta




import pickle # Read/Write with Serialization
import requests # Makes HTTP requests
from io import BytesIO # Use When expecting bytes-like objects
start = dt.datetime.now()


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Classes and images
The number of classes are easily found, looking at the amount of files in the *train_simplified* folder.
Afterwards the amount of images in each class is found in a *for loop* by reading each csv file. The numbers are also used for counting the total amount of images.

In [ ]:
TRAIN_FILES_PATH = '../input/train_simplified/'


trainingFileNameArr = os.listdir(TRAIN_FILES_PATH)
totalClassesCount = len(trainingFileNameArr)
totalImagesCount = 0

print('classes:', totalClassesCount)

#for trainingFileName in trainingFileNameArr:
#  print(trainingFileName)
#  dataset = pd.read_csv(TRAIN_FILES_PATH + trainingFileName, header=0).values
#  print(dataset.shape[0])
#  totalImagesCount += dataset.shape[0]

print(trainingFileNameArr)
print('images: 49707579')
print('Average in each class: 146198.76')

There are 340 classes and 49,707,579 images in total. The amount of images in each class is printed with the class name in the ouput above averaging at 146198.76 per class.
# Printing images from 8 classes.
The first 8 classes has been selected for printing.

In [ ]:
%matplotlib inline
for i in range(8):
    print(trainingFileNameArr[i]);
    path = os.path.join(TRAIN_FILES_PATH,trainingFileNameArr[i]);
    item = pd.read_csv(path);
    item['timestamp'] = pd.to_datetime(item.timestamp);
    item = item.sort_values(by='timestamp',ascending=False)[-20:]
    item['drawing'] = item['drawing'].apply(ast.literal_eval);
    
    item.head()
    
    n = 10
    fig, axs = plt.subplots(2, 10, figsize=(16, 3));
    for i, drawing in enumerate(item.drawing):
        ax = axs[i // n, i % n];
        for x, y in drawing:
            ax.plot(x, -np.array(y), lw=3);
        ax.axis('off');
        
    fig.savefig('item.jpg', dpi=200)
    fig = plt.show();


# Preparing data for network
We first reset the kernel inputs to make sure we don't carry over any variables. This also measn imports must be done again.

In [ ]:
%reset -f 

## Parameter setting
These have been changed a few times. Especially the amount of samples has been increased to use more data, and hopefully increase accuracy. This also requires more VRAM, meaning there is a line in how much data is usable. More data also means longer training time. <br>
To increase the amount of data usable, the batch size is decreased, taking up less VRAM.

In [ ]:
batch_size = 2048
STROKE_COUNT = 196
TRAIN_SAMPLES = 2000
VALID_SAMPLES = 200
TEST_SAMPLES = 150

## The different sizes of samples

TRAIN_SAMPLES = 750,
VALID_SAMPLES = 75,
TEST_SAMPLES = 50
<br>
<br>
TRAIN_SAMPLES = 1000,
VALID_SAMPLES = 100,
TEST_SAMPLES = 75
with 0.5 dropout in last 2
<br>
<br>
TRAIN_SAMPLES = 1200,
VALID_SAMPLES = 120,
TEST_SAMPLES = 100
Best precision
<br>
<br>
TRAIN_SAMPLES = 2000,
VALID_SAMPLES = 200,
TEST_SAMPLES = 150,
Batch size of 2048 necessary instead of 4096
<br>


In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from glob import glob
import gc
gc.enable()
def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
base_dir = os.path.join('..', 'input')
test_path = os.path.join(base_dir, 'test_simplified.csv')

## Creating training data
Here we're creating the training by drawing strokes, training the network to learn the drawings for each stroke making it try to predict from early on, just like the real program does.

In [ ]:
from ast import literal_eval
ALL_TRAIN_PATHS = glob(os.path.join(base_dir, 'train_simplified', '*.csv'))
COL_NAMES = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = literal_eval(raw_strokes) # string->list
    # unwrap the list
    in_strokes = [(xi,yi,i)  
     for i,(x,y) in enumerate(stroke_vec) 
     for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)
def read_batch(samples=5, 
               start_row=0,
               max_rows = 1000):
    """
    load and process the csv files
    this function is horribly inefficient but simple
    """
    out_df_list = []
    for c_path in ALL_TRAIN_PATHS:
        c_df = pd.read_csv(c_path, nrows=max_rows, skiprows=start_row)
        c_df.columns=COL_NAMES
        out_df_list += [c_df.sample(samples)[['drawing', 'word']]]
    full_df = pd.concat(out_df_list)
    full_df['drawing'] = full_df['drawing'].\
        map(_stack_it)
    
    return full_df

## Setup of the different dictionaries and making labels for the data.

In [ ]:
train_args = dict(samples=TRAIN_SAMPLES, 
                  start_row=0, 
                  max_rows=int(TRAIN_SAMPLES*1.5))
valid_args = dict(samples=VALID_SAMPLES, 
                  start_row=train_args['max_rows']+1, 
                  max_rows=VALID_SAMPLES+25)
test_args = dict(samples=TEST_SAMPLES, 
                 start_row=valid_args['max_rows']+train_args['max_rows']+1, 
                 max_rows=TEST_SAMPLES+25)
train_df = read_batch(**train_args)
valid_df = read_batch(**valid_args)
test_df = read_batch(**test_args)
word_encoder = LabelEncoder()
word_encoder.fit(train_df['word'])

## Splitting the data, before feeding it to the network, for training, validation, and testing.

In [ ]:
def get_Xy(in_df):
    X = np.stack(in_df['drawing'], 0)
    y = to_categorical(word_encoder.transform(in_df['word'].values))
    return X, y
train_X, train_y = get_Xy(train_df)
valid_X, valid_y = get_Xy(valid_df)
test_X, test_y = get_Xy(test_df)
print(train_X.shape)

# Network design
The network layout is chosen, as it is a rather small network  it will train faster.<br>
The network is a stroke based Long Short Term Memory network. By using the 1D convolutions the model takes the stroke data and 'preprocesses' it.<br>
After the convolutions two LSTM layers are added followed by two dense layers to classify the data. With the LSTM we're reading the drawing 'stroke for stroke'.

In [ ]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv1D, LSTM, Dense, Dropout
if len(get_available_gpus())>0:
    # https://twitter.com/fchollet/status/918170264608817152?lang=en
    from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances
stroke_read_model = Sequential()
stroke_read_model.add(BatchNormalization(input_shape = (None,)+train_X.shape[2:]))
# filter count and length are taken from the script https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py
stroke_read_model.add(Conv1D(48, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(64, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(96, (3,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = True))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = False))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Dense(512))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Dense(len(word_encoder.classes_), activation = 'softmax'))
stroke_read_model.compile(optimizer = 'adam', 
                          loss = 'categorical_crossentropy', 
                          metrics = ['categorical_accuracy', top_3_accuracy])
stroke_read_model.summary()

# Checkpoints and Callbacks
Setting checkpoints and making callbacks.<br>
This is have some control over the network while training without stopping it to tweak.<br>
By calling the ReduceLROnPlateau, we'll reduce the learning rate if no progression is made in 10 epochs.<br>
Calling EarlyStopping does as the name say, stop early if no progression is made. This is to prevent the network 'running wild' without having any progress at all.


In [ ]:
weight_path="{}_weights.best.hdf5".format('stroke_lstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)


reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
                                   verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
from IPython.display import clear_output
stroke_read_model.fit(train_X, train_y,
                      validation_data = (valid_X, valid_y), 
                      batch_size = batch_size,
                      epochs = 50,
                      callbacks = callbacks_list)
clear_output()

In [ ]:
stroke_read_model.load_weights(weight_path)
lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

# Results
### Variyng results depending on the settings in the network and data size
with 0.5 dropout after the two last layers we reach a precision of 49.8% and 70.6% <br>
with 0.3 dropout after the two last layers we reach a precision of 53.8% and 74.6%<br>
with 0.3 dropout after the two last layers: Accuracy: 59.2%, Top 3 Accuracy 78.6% larger amounts of data
#### with 0.3 dropout after the two last layers: Accuracy: 65.3%, Top 3 Accuracy 83.2%, highest accuracy with largest amount of data
The precision can differ a little due to a regular run through and the commit made to build the notebook.

# Overview
We the create an overview of the precision in every class

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
test_cat = np.argmax(test_y, 1)
pred_y = stroke_read_model.predict(test_X, batch_size = 4096)
pred_cat = np.argmax(pred_y, 1)
plt.matshow(confusion_matrix(test_cat, pred_cat))
print(classification_report(test_cat, pred_cat, 
                            target_names = [x for x in word_encoder.classes_]))

Here we see, that the accuracy differs a lot from class to class. Where some classes has an accuracy above 80% others are below 40%
In the Confusion matrix we some scatters here an there, but in general classification is done correctly.

# Submission and top 3 predictions
We prepare for submission of the results and show three top3 results

In [ ]:
sub_df = pd.read_csv(test_path)
sub_df['drawing'] = sub_df['drawing'].map(_stack_it)

In [ ]:
sub_vec = np.stack(sub_df['drawing'].values, 0)
sub_pred = stroke_read_model.predict(sub_vec, verbose=True, batch_size=2048)

In [ ]:
top_3_pred = [word_encoder.classes_[np.argsort(-1*c_pred)[:3]] for c_pred in sub_pred]

In [ ]:
top_3_pred = [' '.join([col.replace(' ', '_') for col in row]) for row in top_3_pred]
top_3_pred[:3]

# Prediction
We can make some predictions on the test data using the weights from the trained network.

In [ ]:
points_to_use = [5, 15, 20, 30, 40, 50]
points_to_user = [108]
samples = 12
word_dex = lambda x: word_encoder.classes_[x]
rand_idxs = np.random.choice(range(test_X.shape[0]), size = samples)
fig, m_axs = plt.subplots(len(rand_idxs), len(points_to_use), figsize = (24, samples/8*24))
for c_id, c_axs in zip(rand_idxs, m_axs):
    res_idx = np.argmax(test_y[c_id])
    goal_cat = word_encoder.classes_[res_idx]
    
    for pt_idx, (pts, c_ax) in enumerate(zip(points_to_use, c_axs)):
        test_arr = test_X[c_id, :].copy()
        test_arr[pts:] = 0 # short sequences make CudnnLSTM crash, ugh 
        stroke_pred = stroke_read_model.predict(np.expand_dims(test_arr,0))[0]
        top_10_idx = np.argsort(-1*stroke_pred)[:10]
        top_10_sum = np.sum(stroke_pred[top_10_idx])
        
        test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
        lab_idx = np.cumsum(test_arr[:,2]-1)
        for i in np.unique(lab_idx):
            c_ax.plot(test_arr[lab_idx==i,0], 
                    np.max(test_arr[:,1])-test_arr[lab_idx==i,1], # flip y
                      '.-')
        c_ax.axis('off')
        if pt_idx == (len(points_to_use)-1):
            c_ax.set_title('Answer: %s (%2.1f%%) \nPredicted: %s (%2.1f%%)' % (goal_cat, 100*stroke_pred[res_idx]/top_10_sum, word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum))
        else:
            c_ax.set_title('%s (%2.1f%%), %s (%2.1f%%)\nCorrect: (%2.1f%%)' % (word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum, 
                                                                 word_dex(top_10_idx[1]), 100*stroke_pred[top_10_idx[1]]/top_10_sum, 
                                                                 100*stroke_pred[res_idx]/top_10_sum))


Here we see, that predictions made differ very much in precision, proving that the network made is not very precise.
As the sample size for the network is rather low, this can lead to bad learning, as deep learning needs a large database to learn properly.